<a href="https://colab.research.google.com/github/mr-devansh/Lunar-Landing/blob/main/Deep_Q_Learning_for_Lunar_Landing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]
!pip install torch
#gymnasium environments are all involve toy games based around physics control, using box2d based physics and PyGame-based rendering. These environments were contributed back in the early days of OpenAI Gym by Oleg Klimov, and have become popular toy benchmarks ever since. All environments are highly configurable via arguments specified in each environment’s documentation.

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 875.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166

### Importing the libraries

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [ ]:
class Network(nn.Module):

  def __init__(self, state_size, action_size, seed = 42):
    super(Network, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, action_size)

  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)
'''
This code defines a neural network (`Network`) using PyTorch's `nn.Module`
for reinforcement learning tasks. It consists of three fully connected layers
with ReLU activation, mapping input state to output action values,
facilitating learning and decision-making in an agent interacting with an environment.
'''

## Part 2 - Training the AI

### Setting up the environment

In [ ]:
import gymnasium as gym
env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)
'''
This code segment imports the Gymnasium library as `gym`,
creates the LunarLander-v2 environment using `gym.make()`,
retrieves information about the environment such as the state shape,
state size, and the number of available actions, and prints these details.
This setup is crucial for defining the observation and action spaces for the
reinforcement learning agent to interact with the environment effectively.
'''

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [ ]:
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

'''
These are hyperparameters commonly used in reinforcement learning algorithms:

- `learning_rate`: The step size for updating the weights of the neural network during training. Here it's set to 5e-4.

- `minibatch_size`: The number of experiences sampled from the replay memory buffer for each training iteration. It helps stabilize training by reducing the variance in gradient updates. Here it's set to 100.

- `discount_factor`: The factor by which future rewards are discounted in the calculation of the Q-values. It determines the importance of future rewards relative to immediate rewards. Here it's set to 0.99, indicating high importance of future rewards.

- `replay_buffer_size`: The maximum capacity of the replay memory buffer, which stores experiences for training. It's set to 100,000 (1e5) experiences here.

- `interpolation_parameter`: This parameter is used in the soft update of target Q-network parameters towards the local Q-network parameters. It controls the amount of information transferred from the local network to the target network in each update step. Here it's set to 1e-3.'''

### Implementing Experience Replay

In [ ]:
class ReplayMemory(object):

  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones

'''This code defines a `ReplayMemory` class used in reinforcement learning for storing and sampling experiences. Here's a breakdown of its functionality:

- `__init__` method: Initializes the replay memory with a specified capacity. It also determines whether to use CPU or GPU for computations based on the availability of CUDA.

- `push` method: Appends a new experience to the memory. If the memory exceeds its capacity, it removes the oldest experience.

- `sample` method: Samples a batch of experiences from the memory. It randomly selects `batch_size` experiences and returns tensors containing states, next states, actions, rewards, and flags indicating terminal states (dones). It converts experiences to PyTorch tensors and moves them to the appropriate device for computation.

This class facilitates experience replay, a technique used in reinforcement learning to improve sample efficiency and stabilize training by breaking temporal correlations in sequential experiences. It allows the agent to learn from a diverse set of experiences rather than relying solely on recent experiences.'''

### Implementing the DQN class

In [ ]:
class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = Network(state_size, action_size).to(self.device)
    self.target_qnetwork = Network(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

'''This `Agent` class implements a reinforcement learning agent using Deep Q-Learning. Here's an overview:

- **Initializer**:
  - Sets up the agent's device (CPU or GPU), state size, action size, local Q-network, target Q-network, optimizer,
    replay memory, and time step counter.

- **`step` method**:
  - Records experiences (state, action, reward, next_state, done) into the replay memory.
  - Every 4 steps, it samples a batch of experiences from the replay memory and calls the `learn` method to
    update the Q-network.

- **`act` method**:
  - Chooses an action based on the current state using epsilon-greedy policy.
  - With probability epsilon, it chooses a random action; otherwise, it exploits the learned Q-values.

- **`learn` method**:
  - Performs the Q-learning update using a minibatch of experiences sampled from the replay memory.
  - Calculates the expected Q-values using the local Q-network and target Q-values using the target Q-network.
  - Computes the loss between expected and target Q-values and performs backpropagation to update the local Q-network's weights.
  - Also, performs a soft update of target Q-network parameters towards local Q-network parameters.

- **`soft_update` method**:
  - Updates the parameters of the target Q-network using a soft update strategy, blending the local Q-network's parameters
    into the target Q-network's parameters gradually.

This class encapsulates the key components of a Deep Q-Learning agent, including exploration (epsilon-greedy),
experience replay, and target network updates, facilitating learning in the reinforcement learning environment.'''

### Initializing the DQN agent

In [ ]:
agent = Agent(state_size, number_actions)

### Training the DQN agent

In [ ]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break


'''This part of the code is the main training loop where the agent interacts with the environment to learn. Here's what it does:

- **Initialization**:
  - Sets up parameters such as the number of episodes, maximum timesteps per episode, and epsilon-greedy exploration parameters.

- **Training Loop**:
  - Iterates over a specified number of episodes.
  - Resets the environment and initializes the score for the current episode.
  - For each timestep within the episode:
    - Chooses an action using the epsilon-greedy policy based on the current state.
    - Takes a step in the environment with the chosen action and observes the next state, reward, and whether the episode is done.
    - Calls the `step` method of the agent to store the experience and potentially update the Q-network.
    - Updates the current state and accumulates the reward to compute the episode's score.
    - If the episode is done (e.g., the spacecraft crashes or lands), the loop breaks.
  - After each episode, the episode score is appended to a deque of scores for the last 100 episodes to track the moving average score.
  - Epsilon is decayed after each episode.
  - Prints the current episode number and the average score over the last 100 episodes.
  - If the average score over the last 100 episodes reaches 200 (indicating the environment is solved), it stops training,
    saves the trained model's parameters, and exits the loop.

This loop is where the agent learns to navigate the environment by interacting with it, collecting experiences, and updating its Q-network based on the observed rewards and states. It continues until the agent achieves the desired level of performance, in this case, solving the environment.'''

Episode 100	Average Score: -173.85
Episode 200	Average Score: -113.74
Episode 300	Average Score: -44.09
Episode 400	Average Score: 30.40
Episode 500	Average Score: 73.99
Episode 600	Average Score: 169.77
Episode 683	Average Score: 201.98
Environment solved in 583 episodes!	Average Score: 201.98


## Part 3 - Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()